### A. Librerias

In [6]:
import os
import time
import json

import pandas as pd

from utils.utils import get_chrome_driver,stop_chrome_driver
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

### B. Funciones

In [7]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

def clean_files(path, extension='.json'):
    try:
        files = get_files(path, extension)
        for file in files:
            os.remove(file)
        print('Se eliminaron los archivos de la carpeta')
    except:
        pass

### C. Ejecución - Buscar departamentos por distritos

In [8]:
# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False
TIME_SLEEP = 2

# 2. Eliminar todos los archivos de la carpeta
distritos_urls = [
                  ['surquillo','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-surquillo-lima-lima-150141']
                , ['miraflores','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-miraflores-lima-lima-150122']
                , ['barranco','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-barranco-lima-lima-150104']
                , ['breña','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-bre%C3%B1a-lima-lima-150105']
                , ['jesus_maria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-jesus-maria-lima-lima-150113']
                , ['lince', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-lince-lima-lima-150116']
                , ['magdalena', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-magdalena-del-mar-lima-lima-150120']
                , ['pueblo_libre', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-pueblo-libre-lima-lima-150121']
                , ['san_isidro', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-isidro-lima-lima-150131']
                , ['san_miguel', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
                , ['la_victoria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
            ]

# clean_files(os.path.join(os.getcwd(),'originals','1_inicio'))

In [ ]:
for distrito, url in distritos_urls:
    # 0. Eliminar todos los archivos de la carpeta
    clean_files(os.path.join(os.getcwd(),'originals','2_distritos',distrito))
    
    # 1. Iniciar el driver, el servidor proxy y el proxy
    driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

    # 2. Iniciar har
    start_har(proxy, 'nexo')

    # 3. Abrir la página
    driver.get(url)

    # 4. Guardar html
    time.sleep(TIME_SLEEP)
    html = driver.page_source

    with open(os.path.join(os.getcwd(),'originals','1_inicio', distrito + '.html'), 'w') as outfile:
        outfile.write(html)

    # 5. Cerrar el driver
    stop_chrome_driver(driver, server, proxy)

### D. Ejecución - Listar departamentos

In [12]:
urls_list = []
for distrito, x in distritos_urls:
    # 0. Leer el html
    with open(os.path.join(os.getcwd(),'originals','1_inicio','surquillo.html'), 'r') as infile:
        lines = infile.readlines()

    # 1. Buscar la línea que contiene la palabra 'url'
    cadena = ''
    for i in range(len(lines)-1,0,-1):
        if '"url":' in lines[i]:
            cadena = lines[i]
            break
    
    # 2. Buscar las urls
    posiciones_url = []
    posicion = cadena.find('"url":')

    while posicion != -1:
        posiciones_url.append(posicion)
        posicion = cadena.find('"url":', posicion + 1)
    
    # 3. Buscar los project_id
    posiciones_ids = []
    posicion = cadena.find('"project_id":')

    while posicion != -1:
        posiciones_ids.append(posicion)
        posicion = cadena.find('"project_id":', posicion + 1)

    # 4. Buscar links
    for a,b in zip(posiciones_url, posiciones_ids):
        url = ''.join(list(cadena)[a + 7 :b - 2])
        url = url.replace('\x5C','')  
        url = str(url)

        base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
        urls_list.append(base)

len(urls_list)
    

308

### E. Ejecución - Buscar cada departamento

In [14]:
urls_list

['https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2725-eres-fase-2',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2614-nueva-aurora-3',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2313-mate',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2868-residencial-halcones-iii',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2004-v-one',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2241-morar',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-3068-centric',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2385-edificio-altaurora',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2882-capri',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2029-principal-775',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2097-flow-living-apartments',
 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2501-celeste',
 'https://nexoinmobiliario.pe/proyect

In [13]:
# 7. Eliminar los archivos que no tienen datos
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
